In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[2]")\
        .appName("partition_data")\
        .config('spark-master', '7077')\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/31 15:49:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark import SparkContext
from pyspark.sql import functions as func
from pyspark.sql import *

In [3]:
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("dfs.block.size", "10m")

In [4]:
df = spark.read.csv("hdfs://namenode:9000/bigdataproject/raws_data/data.csv",escape = "\"", header = True, inferSchema=True,multiLine=True)

In [5]:
df = df.select("*").withColumn("id", func.monotonically_increasing_id())

In [6]:
PARTITION_SIZE = 500
df = df.withColumn("group", func.floor(func.col("id")/PARTITION_SIZE))

In [7]:
df.schema

StructType(List(StructField(Mức giá,DoubleType,true),StructField(Diện tích,DoubleType,true),StructField(Loại tin đăng,StringType,true),StructField(Địa chỉ,StringType,true),StructField(Mặt tiền,DoubleType,true),StructField(Đường vào,DoubleType,true),StructField(Hướng ban công,StringType,true),StructField(Số tầng,IntegerType,true),StructField(Số phòng ngủ,IntegerType,true),StructField(Số toilet,IntegerType,true),StructField(Pháp lý,StringType,true),StructField(Tên dự án,StringType,true),StructField(Chủ đầu tư,StringType,true),StructField(Ngày đăng,StringType,true),StructField(Ngày hết hạn,StringType,true),StructField(Mã tin,IntegerType,true),StructField(Phòng ngủ,StringType,true),StructField(Hướng nhà,StringType,true),StructField(Loại tin,StringType,true),StructField(Tỉnh,StringType,true),StructField(Quận/Huyện,StringType,true),StructField(Thời gian đăng,StringType,true),StructField(id,LongType,false),StructField(group,LongType,true)))

In [9]:
import time
steps = df.select("group").distinct().sort("group").limit(2).collect()
for step in steps[:]:
    start = time.time()
    _df = df.where(f"group = {step[0]}")
    #by adding coalesce(1) we save the dataframe to one file
    _df.coalesce(1).write.mode("append").option("header", "true").csv("hdfs://namenode:9000/bigdataproject/streaming_data")
    time.sleep(30+start-time.time())